In [17]:
import pandas as pd
import datetime
import numpy as np
import re

load item_code and clean

In [18]:
item_code = pd.read_csv('C:/Users/kwang648/Downloads/call_report_item_new.csv')

In [19]:
item_code = item_code.drop(['Notes','Description','citation'],axis=1)

In [20]:
#change end period to 2262-04-11 indicating the variable is currently in use
item_code['end'] = item_code['end'].apply(lambda x: 22620411 if x==99991231 else x)
#convert begin and end period to DateTime type
item_code['begin']=pd.to_datetime(item_code['begin'],format='%Y%m%d')
item_code['end']=pd.to_datetime(item_code['end'],format='%Y%m%d')
item_code = item_code.dropna(subset=['var_name'])

extract variable names

In [21]:
var_list = []
for code in item_code['item_code']:
    list = re.findall("[a-zA-Z0-9]+",code)
    for item in list:
        if item not in var_list:
            var_list.append(item)

Load call report: to reduce memory, only load the columns in the var_list, and skip missing columns in call report

In [22]:
total_list = pd.read_csv('C:/Users/kwang648/Downloads/banking data/call_2000-2010.csv', index_col=0, nrows=0).columns.tolist()
call_report_varlist = [element for element in var_list if element in total_list]

In [23]:
call_report = pd.read_csv('C:/Users/kwang648/Downloads/banking data/call_2000-2010.csv',encoding='latin-1',usecols = call_report_varlist)
call_report = call_report.dropna(subset=['DATE'])
call_report['DATE'] = pd.to_datetime(call_report['DATE'],format='%Y-%m-%d')

C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


define variables

In [24]:
def handle_missing_code(code):
    code_list = re.findall("[a-zA-Z0-9]+",code)
    for item in code_list:
        if item not in call_report.columns:
            print(item+' is not available')
            return True
    return False

In [25]:
for varname in item_code['var_name'].unique():
    if varname=='DATE':
        continue
    #find begin and end date
    begin = item_code[item_code['var_name']==varname]['begin']
    end = item_code[item_code['var_name']==varname]['end']
    code = item_code[item_code['var_name']==varname]['item_code']
    #handle variable definition with multiple periods
    i = 1
    while i<=len(begin):
        print(varname + ": " + code.iloc[i-1])
        if handle_missing_code(code.iloc[i-1]) ==True:
            i=i+1
            continue
        if len(code.iloc[i-1])==8:
            call_report.loc[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1]),varname] = call_report[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1])][code.iloc[i-1]]
        else:
            call_report.loc[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1]),varname] = call_report[(call_report['DATE']>=begin.iloc[i-1]) &(call_report['DATE']<=end.iloc[i-1])].eval(code.iloc[i-1])
        i=i+1

RSSD_ID: RSSD9001
CERT: RSSD9050
REG_HIGH_HOLD: RSSD9348
LOCATION: RSSD9210
ZIPCODE: RSSD9220
STATE: RSSD9200
assets: RCFD2170
assets: RCFD2170
reloans: RCFD1410
cash: RCFD0010
cash: RCFD0071+RCFD0081
persloans: RCFD1975
agloans: RCFD1590
subordinateddebt: RCFD3200
equity: RCFD3210
equity: RCFD8727+RCFD8731
equity: RCFD8727
demanddep: RCON2210
transdep: RCON2215
brokereddep: RCON2365
brokereddep: RCON2365 
timedepge100k: RCON2604
timedepge100k: RCONJ474+RCONJ473
cdge100k: RCON6645
timedeple100k: RCON6648
timedepge250k: RCONJ474
timedeple250k: RCONJ473+RCON6648
ciloans: RCFD1600
ciloans: RCFD1766
loans: RCFD1400+RCFD2165
loans: RCFD1400
loans: RCFD1400
loans: RCFD2125+RCFD2165 
loans: RCFD2125 
loansnet: RCFD2122-RCFD3123
fedfundsrepoasset: RCFD1350
fedfundsrepoasset: RCFDB987+RCFDB989
tradingassets: RCFD2146
tradingassets: RCFD3545
securities: RCFD0400+RCFD0600+RCFD0900+RCFD0950
RCFD0900 is not available
securities: RCFD0390
securities: RCFD1754+RCFD1773
securities: RCFD0390 + RCFD2146

derivhedg: RCFD8725
currency: RCFD8726+RCFD8720
currency: RCFD8726
commderiv: RCFD8728+RCFD8732
commderiv: RCFD8728
derivtrade: RCFDA126+RCFDA127+RCFD8723+RCFD8724
netincome: RIAD4340
maturitygap: RCONA570+RCONA571+RCONA564+RCONA565+RCONA549+RCONA550+RCONA555+RCONA556+RCFD1350+RCFD2155-(RCONA579+RCONA580+RCONA584+RCONA585-RCFD2800+RCFD2850+RCFD2920)/RCFD2170
nonperfassets: (RCFD1407+RCFD1403)/RCFD2170
liquidity: (RCFD0010+RCFD1350+RCFD0390+RCFD2146)/RCFD2170
liquidity: (RCFD0010+RCFD1350+RCFD1754+RCFD1773)/RCFD2170
liquidity: RCFD0400+RCFD0600+RCFD0900+RCFD0380+RCFD1350
RCFD0900 is not available
liquidity: RCFD0390+RCFD1350+RCFD2146
liquidity: RCFD1350+RCFD1754+ RCFD3545 
capitalization: RCFD3210/RCFD2170 


In [26]:
call_report

,DATE,RCFDA247,RCFDA248,RCFDA549,RCFDA550,RCFDA551,RCFDA552,RCFDA553,RCFDA554,RCFDA555,...,consumerloans,derivhedg,currency,commderiv,derivtrade,netincome,maturitygap,nonperfassets,liquidity,capitalization
0,2000-03-31,11425.0,591.0,99.0,365.0,1410.0,5829.0,11937.0,426.0,44.0,...,NaN,0.0,0.0,0.0,0.0,221.0,NaN,0.005617,0.413697,0.167098
1,2000-03-31,2904.0,454.0,0.0,454.0,3898.0,4761.0,217.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,69.0,NaN,0.002321,0.463111,0.139060
2,2000-03-31,3179.0,296.0,997.0,0.0,0.0,5039.0,25515.0,2574.0,1612.0,...,NaN,0.0,0.0,0.0,0.0,166.0,NaN,0.000547,0.752774,0.150699
3,2000-03-31,1560.0,198.0,0.0,198.0,738.0,1483.0,1582.0,0.0,40.0,...,NaN,0.0,0.0,0.0,0.0,12.0,NaN,0.008773,0.564526,0.117864
4,2000-03-31,20655.0,1752.0,500.0,837.0,2574.0,5329.0,8645.0,0.0,307.0,...,NaN,0.0,0.0,0.0,0.0,327.0,NaN,0.000767,0.314641,0.124718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126605,2003-06-30,781.0,0.0,0.0,0.0,0.0,5050.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1146.0,NaN,0.000000,NaN,0.697620
126606,2003-06-30,71.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,118.0,0.0,0.0,0.0,0.0,-428.0,NaN,0.000000,NaN,0.610895
126607,2003-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.000000,0.305312,NaN
126608,2003-06-30,5437.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2118.0,0.0,0.0,0.0,0.0,10883.0,NaN,0.268266,NaN,0.403991
